In [162]:
from PIL import Image, ImageEnhance
import numpy as np 
import os
import cv2
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
# np.save('example_1', a)
# b = np.load('example/example_1.npy')
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import random
import tensorflow as tf
from keras.models import load_model
#import keras

In [175]:
image_folder = "/Users/alexeyvaganov/doc/files/color_matcher/BW"
image_massiv = "/Users/alexeyvaganov/doc/files/color_matcher/temp/"

In [2]:
# Обходим все файлы в папке Y
# Нам надо сделать массив, в котором в кажой строке будет два элемента
# в первом элементе массива искомая фотография
# во втором элемент массива измененная фотография
# из каждой фотографии делаем N измененых фотографий в зависимости от параметров
# сохраняем массив на диск

In [164]:
array_transform_nastr = [
    [1.5,0.5,0.5],
    [1.5,1,0.5],
    [1,1,0.8],
    [1,1,2],
    [2,1.2,0],
    [2,1.2,0.1]
]

# array_transform_nastr = [
#     [0.5,0.5,0.5],
#     [1.0,1.0,1.0],
#     [0.2,0.2,0.2],
#     [1,1,2],
#     [0.8,1.2,0],
#     [2,1.2,0.1]
# ]

In [165]:
def show_image(img_file):
    img = Image.open(img_file)
    img.show()

In [166]:
def transform_image(img_pil,brightness,contrast,color):
    if (brightness != 0):
        enhancer = ImageEnhance.Brightness(img_pil)
        img_pil = enhancer.enhance(brightness)
    if (contrast != 0):
        enhancer = ImageEnhance.Contrast(img_pil)
        img_pil = enhancer.enhance(contrast)
    if (color != 0):
        enhancer = ImageEnhance.Color(img_pil)
        img_pil = enhancer.enhance(color) 
    return img_pil

In [167]:
array_image = []
for root, dirs, files in os.walk(image_folder):
    for file in files:
        if file == ".DS_Store":
            continue
        img_pil = Image.open(root + "/" + file)
        img_pil = img_pil.convert('RGB')
        img_pil_lab = rgb2lab(img_pil)
        #img_cv2_ish = cv2.imread(root + "/" + file)
        for x in array_transform_nastr:
            img_ = transform_image(img_pil,x[0],x[1],x[2])
            #img_.save("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")
            #img_cv2_transf = cv2.imread("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")
            img_lab = rgb2lab(img_)
            #img_pil = rgb2lab(img_pil)
            array_image.append([np.array(img_lab),np.array(img_pil_lab)])
        
array_full_image = np.array(array_image) 
array_full_image.shape

(18, 2, 960, 1440, 3)

In [24]:
#img_show = array_full_image[0,0]
#imsave("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg", lab2rgb(array_full_image[1,0]))
#img_show.save("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg")

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [172]:
#np.savez_compressed(image_massiv, a=np_array)

In [168]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 2)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
#model.compile(optimizer='rmsprop', loss='mse', metrics=[keras.metrics.SparseCategoricalAccuracy()])
#model.compile(optimizer='rmsprop', loss='mse')
model.compile(optimizer='Adadelta', loss='mse')

In [218]:
#imsave("/Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg", np_array_x_y[0,0])

/Users/alexeyvaganov/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: /Users/alexeyvaganov/doc/files/color_matcher/temp/geeks.jpg is a low contrast image
  """Entry point for launching an IPython kernel.


In [169]:
def crop_image(array_x_y,a,b):
    h=0
    while h <= a.shape[0]-56:
        w=0
        while w <= a.shape[1]-56:
            x_ = a[h:h+56,w:w+56]
            y_ = b[h:h+56,w:w+56]
            array_x_y.append([x_,y_])
            w+=56
        h+=56

In [170]:
array_x_y = []
i=0
x_sgape = array_full_image.shape[0]
while i < x_sgape:
    crop_image(array_x_y,array_full_image[i,0],array_full_image[i,1])
    i+=1
array_crop_image = np.array(array_x_y) 
array_crop_image = 1.0/255*array_crop_image
array_crop_image.shape

(7650, 2, 56, 56, 3)

In [171]:
X_ = array_crop_image[:,0]
Y_ = array_crop_image[:,1]
X_ = X_[:,:,:,1:]
Y_ = Y_[:,:,:,1:]

In [ ]:
model = load_model(image_massiv + 'Velvia_19_8_15.h5')

In [172]:
model.fit(X_,Y_,batch_size=32, epochs=10)

Epoch 1/10
240/240 [==============================] - 41s 165ms/step - loss: 3.1244e-05
Epoch 2/10
240/240 [==============================] - 40s 168ms/step - loss: 2.4977e-05
Epoch 3/10
240/240 [==============================] - 53s 219ms/step - loss: 2.1021e-05
Epoch 4/10
240/240 [==============================] - 47s 195ms/step - loss: 1.8182e-05
Epoch 5/10
240/240 [==============================] - 55s 227ms/step - loss: 1.6248e-05
Epoch 6/10
240/240 [==============================] - 43s 180ms/step - loss: 1.4887e-05
Epoch 7/10
240/240 [==============================] - 50s 210ms/step - loss: 1.4043e-05
Epoch 8/10
240/240 [==============================] - 45s 185ms/step - loss: 1.3448e-05
Epoch 9/10
240/240 [==============================] - 41s 171ms/step - loss: 1.3273e-05
Epoch 10/10
240/240 [==============================] - 42s 174ms/step - loss: 1.3069e-05


# Предсказание

In [173]:
Obr = 1

In [179]:
#array_img_test = []
file_name = "IMG_2269.jpg"
test_image_path = image_massiv + file_name
img_test = Image.open(test_image_path)
img_test = img_test.convert('RGB')
#array_img_test.append(img_test) 
img_test = rgb2lab(img_test)
array_full_img_test = np.array(img_test) 
array_full_img_test.shape

(3088, 2316, 3)

In [180]:
new_img = np.zeros((array_full_img_test.shape[0], array_full_img_test.shape[1], 3))
h=0
while h <= array_full_img_test.shape[0]-56:
    w=0
    while w <= array_full_img_test.shape[1]-56:
        # predict
        x_iz = array_full_img_test[h:h+56,w:w+56]
        x_ = x_iz[:,:,1:]
        x_ = 1.0/255*x_
        x_ = x_.reshape(1,56,56,2)
        pred_y = model.predict(x_)
        
        cur = np.zeros((56, 56, 3))
        cur[:,:,0] = x_iz[:,:,0]
        pred_y = pred_y[0]* 256
        cur[:,:,1:] = pred_y
        
        new_img[h:h+56,w:w+56] = cur
        
        w+=56
    h+=56

In [181]:
imsave(image_massiv + str(Obr) + file_name, lab2rgb(new_img))
Obr += 1

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [120]:
model.save(image_massiv + Velvia_19_8_4.h5)

# Прочее

In [35]:
img_pred = array_full_img_test[0:56,0:56]
img_pred_col = img_pred[:,:,1:]
img_pred_col = 1.0/255*img_pred_col
img_pred_col = img_pred_col.reshape(1,56,56,2)

In [15]:
img_lab = img_pred[:,:,:1]
print (img_lab.shape)
print (img_lab)

(56, 56, 1)
[[[87.62802886]
  [87.62802886]
  [87.27331251]
  ...
  [87.62802886]
  [87.68525276]
  [87.50052494]]

 [[67.2156848 ]
  [66.84179228]
  [66.84179228]
  ...
  [68.50146286]
  [68.56967781]
  [68.56967781]]

 [[67.18258232]
  [67.18258232]
  [67.18258232]
  ...
  [68.59121952]
  [68.59121952]
  [68.59121952]]

 ...

 [[70.22792326]
  [70.22792326]
  [70.59680428]
  ...
  [71.52757235]
  [71.52757235]
  [71.52757235]]

 [[70.22792326]
  [70.22792326]
  [70.59680428]
  ...
  [71.52757235]
  [71.52757235]
  [71.52757235]]

 [[70.22792326]
  [70.22792326]
  [70.59680428]
  ...
  [71.52757235]
  [71.52757235]
  [71.52757235]]]


In [36]:
pr_mod = model.predict(img_pred_col)

In [17]:
print(pr_mod_f[0].shape)
print(img_lab.shape)

NameError: name 'pr_mod_f' is not defined

In [82]:
cur = np.zeros((56, 56, 3))
cur[:,:,0] = img_pred[:,:,0]
prrr = pr_mod[0]* 256
cur[:,:,1:] = prrr

In [83]:
cur[0,0]

array([208.        , 157.64634705, 180.79605103])

In [74]:
pr_mod_f = pr_mod * 256
it_img = pr_mod_f[0][:,:,] + img_lab[:,:,]
it_img.shape

(56, 56, 2)

In [115]:
#new_img = np.zeros((array_full_img_test.shape[0], array_full_img_test.shape[1], 3))

In [ ]:
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

In [374]:
# Image transformer
batch_size = 10
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

In [358]:
# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

(90972, 2, 50, 50, 3)

In [ ]:
# Train model      
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=1, steps_per_epoch=10)

In [326]:
# Import map images into the lab colorspace
def image_to_lab(image):
    X = rgb2lab(1.0/255*image)[:,:,0]
    #X = X.reshape(X.shape+(1,))
    X = X.reshape(1,50,50,1)
    return X

In [323]:
wqw2w = image_to_lab(np_array_x_y[0,0])

In [324]:
wqw2w.shape

(1, 50, 50, 1)

In [342]:
array_x_lab = []
array_y_lab = []
i = 0
#while i < np_array_x_y.shape[0]:
while i < 1:
    lab_x = image_to_lab(np_array_x_y[i,0])
    array_x_lab.append(lab_x)
    lab_y = image_to_lab(np_array_x_y[i,1])
    array_y_lab.append(lab_y)
    i+=1

In [334]:
X = array_x_lab[:10]
Y = array_y_lab[:10]

In [343]:
X = np.array(X) 
Y = np.array(Y) 

In [344]:
Y[0].shape

(1, 50, 50, 1)

In [345]:
model.fit(x=X, 
    y=Y,
    batch_size=10,
    epochs=50)

Epoch 1/50


ValueError: in user code:

    /Users/alexeyvaganov/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:784 train_function  *
        return step_function(self, iterator)
    /Users/alexeyvaganov/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:774 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/alexeyvaganov/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1261 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/alexeyvaganov/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2725 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/alexeyvaganov/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3337 _call_for_each_replica
        return fn(*args, **kwargs)
    /Use

In [273]:
d.shape

(50, 50, 1)